In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import contractions
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreenidhihegde/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreenidhihegde/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz

## Read Data

In [2]:
 """
 Here we are reading the data from tsv file as pandas dataframe
 We use ''\t' to seperate columns by a tab and "error_bad_lines = False" to drop bad lines from the DataFrame
 
 """  
text_data = pd.read_csv("data.tsv",error_bad_lines = False, sep = '\t')
text_data.head()

/Users/shreenidhihegde/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310:

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,37000337,R3DT59XH7HXR9K,B00303FI0G,529320574,Arthur Court Paper Towel Holder,Kitchen,5.0,0.0,0.0,N,Y,Beautiful. Looks great on counter,Beautiful. Looks great on counter.,2015-08-31
1,US,15272914,R1LFS11BNASSU8,B00JCZKZN6,274237558,Olde Thompson Bavaria Glass Salt and Pepper Mi...,Kitchen,5.0,0.0,1.0,N,Y,Awesome & Self-ness,I personally have 5 days sets and have also bo...,2015-08-31
2,US,36137863,R296RT05AG0AF6,B00JLIKA5C,544675303,Progressive International PL8 Professional Man...,Kitchen,5.0,0.0,0.0,N,Y,Fabulous and worth every penny,Fabulous and worth every penny. Used for clean...,2015-08-31
3,US,43311049,R3V37XDZ7ZCI3L,B000GBNB8G,491599489,Zyliss Jumbo Garlic Press,Kitchen,5.0,0.0,1.0,N,Y,Five Stars,A must if you love garlic on tomato marinara s...,2015-08-31
4,US,13763148,R14GU232NQFYX2,B00VJ5KX9S,353790155,"1 X Premier Pizza Cutter - Stainless Steel 14""...",Kitchen,5.0,0.0,0.0,N,Y,Better than sex,Worth every penny! Buy one now and be a pizza ...,2015-08-31


## Keep Reviews and Ratings

In [3]:
    #We are using dropna to look for missing values in the rows which has no review body and drop the corresponding rows.
    
    text_data.dropna(subset = ['review_body'], inplace= True)
    text_data.dropna(subset = ['star_rating'], inplace= True)
    
    
    # We Keep only the reviews and ratings of the initial data
    text_data = text_data[["review_body","star_rating"]]
    
    # We are including 3 sample reviews with the corresponding rating
    print ("\nxxxxx Three sample reviews with corresponding ratings xxxxx\n")
    print (text_data.sample(3))
    
    # We are reporting statistics of the ratings
    star_counts = text_data["star_rating"].value_counts()
    print("\n====== Statistics of the ratings before preprocessing =======\n")
    print(star_counts)


xxxxx Three sample reviews with corresponding ratings xxxxx

                                               review_body  star_rating
3959067  The quality is high, the Capacity is suitable....          5.0
4044080  Looks very nice and is a good size, but I find...          3.0
4354222  I have owned a ditcenaid for 18 years and have...          5.0

====== Statistics of the ratings before preprocessing =======

5.0    3124595
4.0     731701
1.0     426870
3.0     349539
2.0     241939
Name: star_rating, dtype: int64


# Labelling Reviews:
## The reviews with rating 4,5 are labelled to be 1 and 1,2 are labelled as 0. Discard the reviews with rating 3'

In [4]:
# Using np.where we are putting conditions to label the ratings >=4 as 1 and ratins <=2 as 0. 
#The remainign ratings which is labelled as 3 will be labelled as -1.

text_data['label'] = np.where(text_data["star_rating"] >=4,1, np.where(text_data["star_rating"] <= 2,0,-1))
rating_count = text_data['label'].value_counts()

#We are getting the counts of all review labels before removing the reviews with rating 3
print("\n====== Counts of all review labels before removing the reviews with rating 3 =======\n")
print(rating_count)

#Discarding the reviews with rating 3
text_data = text_data[text_data["label"]!= -1]

#We are getting the counts of all review labels after removing the reviews with rating 3
print("\n====== Counts of all review labels after removing the reviews with rating 3 =======\n")
print(text_data['label'].value_counts())



====== Counts of all review labels before removing the reviews with rating 3 =======

 1    3856296
 0     668809
-1     349539
Name: label, dtype: int64

====== Counts of all review labels after removing the reviews with rating 3 =======

1    3856296
0     668809
Name: label, dtype: int64


 ## We select 200000 reviews randomly with 100,000 positive and 100,000 negative reviews.



In [5]:
#We select sample of 100,000 positive and 100,000 negative reviews
negative = text_data.label[text_data.label.eq(0)].sample(100000).index
positive = text_data.label[text_data.label.eq(1)].sample(100000).index

#We combine both the selected samples
text_data = text_data.loc[negative.union(positive)]
print(text_data)

# We are getting mean of each row based on characters in each row and then finding the mean of all the rows
review_mean = text_data.review_body.apply(lambda x : np.mean(len(str(x)))).mean()
# print 3 sample reviews
print(review_mean)                                                                                   
                                          
                                                             
                                                             




                                               review_body  star_rating  label
26       I was having a hard time opening small lids on...          5.0      1
54       The wood broke after a week of use. The metal ...          1.0      0
86       Lite weight compared to my old stanly thermos....          5.0      1
88       I have tried to get this canner to work for ma...          1.0      0
92       The lid doesn't hold the container.  It only w...          2.0      0
...                                                    ...          ...    ...
4874673  I don't go for kitchen gadgets. Many of them t...          5.0      1
4874691  We've tried a few of these because they look l...          1.0      0
4874726  Though Maxim is a fine company, this particula...          2.0      0
4874730  We purchased a Henckels Four-Star set about te...          5.0      1
4874734  So far, my wife and I have made a batch of min...          5.0      1

[200000 rows x 3 columns]
322.10973


# Data Cleaning

## Convert the all reviews into the lower case.

In [6]:
#We use str.lower() to convert all the characters into lower characters
text_data["review_body"] = text_data["review_body"].str.lower()


 ## Remove the HTML and URLs from the reviews

In [7]:
#We use Beatiful soup to remove all the HTML Tags from the dataframe
text_data["review_body"] = text_data["review_body"].apply(lambda x: BeautifulSoup(str(x)).get_text())

# We are here removing the URLs from the reviews
text_data["review_body"]= text_data["review_body"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()


/Users/shreenidhihegde/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.amazon.com/review/create-review/ref=cm_cr_dp_wrt_summary?ie=utf8&asin=b00xp0d9p0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/shreenidhihegde/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.amazon.com/review/create-review?ie=utf8&asin=b00si28464" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/shreenidhihegde/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "http://www.consumerreports.org/cro/magazine-archiv

## remove non-alphabetical characters

In [8]:
#First we remove all the words starts with digits
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(r"[^\D']+", " ", str(x), flags=re.UNICODE))

#Next we remove all the words which starts with non alphabetic characters
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(r"[^\w']+", " ", str(x), flags=re.UNICODE))

# print(text_data.sample(10))


## Remove the extra spaces between the words

In [9]:
#We remove the extra spaces from the dataset
text_data["review_body"] = text_data.review_body.replace('\s+', ' ', regex=True)
text_data["review_body"]

26         i was having a hard time opening small lids on...
54         the wood broke after a week of use the metal l...
86         lite weight compared to my old stanly thermos ...
88         i have tried to get this canner to work for ma...
92         the lid doesn't hold the container it only wor...
                                 ...                        
4874673    i don't go for kitchen gadgets many of them ta...
4874691    we've tried a few of these because they look l...
4874726    though maxim is a fine company this particular...
4874730    we purchased a henckels four star set about te...
4874734    so far my wife and i have made a batch of mint...
Name: review_body, Length: 200000, dtype: object

## perform contractions on the reviews.

In [10]:
#We perform contractions using contractions.fix

def contractionfunction(i):
    i = i.apply(lambda x: contractions.fix(x))
    return i

text_data["review_body"] = contractionfunction(text_data["review_body"])

    

# Pre-processing

## remove the stop words 

In [11]:
# We are removing stop words 

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

text_data["review_body"] = text_data["review_body"].apply(lambda x: " ".join([i for i in x.split() if i not in stop_words]))





 

## perform lemmatization  

In [75]:
#We perform lemmatization on the data

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
text_data["review_body"] = text_data["review_body"].apply(lambda x: " ".join(lemmatizer.lemmatize(i) for i in x.split()))
text_data["review_body"]

review_mean = text_data.review_body.apply(lambda x : np.mean(len(str(x)))).mean()
print(review_mean)


print(text_data["review_body"].sample(3))




207.01535
1514186    bought gift christmas present brother. loved c...
4187605    worst dishwasher! 2 years, started getting \\"...
4158822    sugar bowl perfect. smaller size expected, hol...
Name: review_body, dtype: object


# TF-IDF Feature Extraction

In [12]:
# We use train_test_split from sklearn to split the data into 80% training and 20% testing sets
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test = train_test_split(text_data["review_body"], text_data["label"], test_size=0.33, random_state=42)

#We use ....
vectorizer = TfidfVectorizer(max_features=2000, min_df=0, max_df=0.7)
Xtrain = vectorizer.fit_transform(X_train)
Xtest = vectorizer.transform(X_test)
    

In [13]:
# we standardize the data using StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

#Create the instance
sc = StandardScaler(with_mean=False)

#We fit the scaler to the training feauture set only
sc.fit(Xtrain)

#Scale or Transform the training and the testing tests using the scaler that was fitted to training data
Xtrain_std = sc.transform(Xtrain)
Xtest_std = sc.transform(Xtest)

In [58]:
print(Xtest_std)

  (0, 1987)	48.15104885487249
  (0, 1876)	4.55769444080691
  (0, 1385)	25.897171026031252
  (0, 999)	7.105163181861919
  (0, 817)	10.77701893771047
  (0, 592)	6.0632592242074015
  (0, 584)	8.176619328694843
  (0, 448)	6.5091803440489695
  (1, 1988)	3.5104122951121415
  (1, 1978)	2.24293689323711
  (1, 1876)	3.0038967417704385
  (1, 1857)	10.679589913550817
  (1, 1796)	2.401934242251563
  (1, 1480)	5.052611058356096
  (1, 1463)	19.943686312547175
  (1, 1328)	1.804421062889562
  (1, 993)	11.69426902705059
  (1, 769)	6.125781588674803
  (1, 762)	1.2372824030983103
  (1, 750)	15.94564098916578
  (1, 683)	15.931870701855095
  (1, 590)	10.86683832943603
  (1, 538)	5.7316319392821535
  (1, 524)	16.078139774218602
  (1, 448)	4.2900869904791525
  :	:
  (65999, 1967)	1.4543168885367872
  (65999, 1957)	6.060530221728638
  (65999, 1954)	11.407722182481107
  (65999, 1926)	4.062192754228056
  (65999, 1876)	2.7485866147112805
  (65999, 1850)	10.16308785734334
  (65999, 1796)	2.1977867001441207
  (659

# Perceptron

In [14]:
#implementation of perceptron
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
ppn = Perceptron(max_iter=40, eta0=0.1, random_state=0)

# Fit the model to the standardized data
ppn.fit(Xtrain_std, y_train)

# Apply the trained perceptron on the  data to make predicts for the test data
y_pred = ppn.predict(Xtest_std)

#We measure the performance using the "accuracy_score"
print('Accuracy:{0:.2f}%' .format(accuracy_score(y_test, y_pred)*100))

Accuracy:84.78%


# SVM

In [15]:
#implementation of SVM
from sklearn import svm

#Create a Classifier for svm
clf = svm.LinearSVC() # We are using Linear Kernel

#Train the model using the training sets
clf.fit(Xtrain_std, y_train)

#Apply the trained svm on Xtrain_std data to make predictions for the test data
y_pred = clf.predict(Xtest_std)

print('Accuracy:{0:.2f}%' .format(accuracy_score(y_test, y_pred)*100))

Accuracy:84.99%


/Users/shreenidhihegde/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(Xtrain_std,y_train)

#Apply the trained Logistic Regression on Xtrain_std data to make predictions for the test data
y_pred=logreg.predict(Xtest_std)

from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix
print('Accuracy:{0:.2f}%' .format(accuracy_score(y_test, y_pred)*100))

Accuracy:89.28%


# Naive Bayes

In [17]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

#Create a Gaussian Classifier
model = MultinomialNB()

# Train the model using the training sets
model.fit(Xtrain_std, y_train)

#Predict Output
y_pred = model.predict(Xtest_std) # 0:Overcast, 2:Mild
print(y_pred)
print('Accuracy:{0:.2f}%' .format(accuracy_score(y_test, y_pred)*100))

[0 1 0 ... 0 1 1]
Accuracy:84.88%
